In [1]:

import string
import nltk
import re
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
string.punctuation
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer

porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
def word_extraction(sentence):    
   ignore = ['a', "the", "is"]    
   words = re.sub("[^\w]", " ",  sentence).split()    
   cleaned_text = [w.lower() for w in words if w not in ignore]    
   return cleaned_text

In [3]:
def tokenize(sentence):    
   words = re.sub("[^\w]", " ",  sentence).split()    
   words = [w.lower() for w in words if w not in stopwords]  
   tokens = [i for i in words if not i.isdigit()]
   lemm_text_tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]    
   return lemm_text_tokens

In [4]:
print(tokenize('Hey! I am a boy. from room 3'))

['hey', 'i', 'boy', 'room']


In [6]:
filename = '/content/en-train.txt'
with open(filename,'r') as file_ptr:
  data = file_ptr.read()
  sentencelist=sent_tokenize(data)
  tokens=[]
  for string in sentencelist:
    tokens.extend(tokenize(string))
print(tokens[:100])




['umm', 'california', 'cargo', 'ship', 'i', 'dont', 'care', 'article', 'say', 'the', 'california', 'cargo', 'ship', 'how', 'i', 'prepare', 'old', 'exterior', 'wall', 'painting', 'how', 'i', 'prepare', 'exterior', 'concrete', 'wall', 'paint', 'the', 'man', 'playing', 'guitar', 'a', 'man', 'playing', 'guitar', 'it', 'final', 'test', 'delivering', 'missile', 'armed', 'force', 'state', 'radio', 'said', 'last', 'test', 'missile', 'delivered', 'armed', 'force', 'this', 'fully', 'answer', 'question', 'certainly', 'one', 'consideration', 'i', 'phd', 'student', 'computational', 'science', 'interdisciplinary', 'major', 'spanning', 'mathematics', 'computer', 'science', 'engineerning', 'a', 'longed', 'haired', 'cat', 'eye', 'closed', 'a', 'close', 'cat', 'eye', 'closed', 'the', 'venture', 'owns', 'five', 'oil', 'refinery', 'filling', 'station', 'russia', 'ukraine', 'tnk', 'bp', 'six', 'oil', 'producing', 'unit', 'five', 'refinery', 'filling']


In [7]:
print(len(tokens))

184304


In [8]:
print(len(set(tokens)))

15607


In [13]:

vocab = list(set(tokens))
vocab_index_dict=dict()
word_count = len(vocab)
for i,word in enumerate(vocab):                                 
    vocab_index_dict[word] = i
def generate_bow(sentence):        
  # print("Word List for Document \n{0} \n".format(vocab))       
  bag_vector = np.zeros(word_count+1)
  words = tokenize(sentence)        
  for w in words:
    idx = vocab_index_dict.get(w,word_count)           
    bag_vector[idx]=1                           
  print("{0}\n{1}\n".format(sentence,np.array(bag_vector)))
  return bag_vector

In [14]:
generate_bow('Umm the California was a cargo ship I dont care what the article says.')

Umm the California was a cargo ship I dont care what the article says.
[0. 0. 0. ... 0. 0. 0.]



array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
from numpy import dot
from numpy.linalg import norm
def get_cosine_similarity(s1,s2):
  a = generate_bow(s1)
  b = generate_bow(s2)
  cos_sim = dot(a, b)/(norm(a)*norm(b))
  # print("Cosine:", cos_sim*5) 
  return cos_sim*5


In [ ]:
with open(filename) as f:
    lines = [line.rstrip('\n') for line in f]
s1_lines,s2_lines,actual_scores,predicted_scores = list(),list(),list(),list()
for line in lines:
  s1,s2,score = line.split('\t')
  s1_lines.append(s1)
  s2_lines.append(s2)
  actual_scores.append(eval(score))
  y =  get_cosine_similarity(s1,s2)
  predicted_scores.append(y)

In [24]:
from scipy.stats import pearsonr
actual_scores = [int(i) for i in actual_scores]
pcc, p_value = pearsonr(actual_scores,predicted_scores)
# Print the PCC
print("PCC: {:.3f}".format(pcc))

PCC: 0.627
